# Imports

In [1]:
# ignore deprecation warnings ('safe' as long as we don't update packages)
from warnings import filterwarnings
filterwarnings("ignore")

from math import sqrt, log, inf
import copy

import gym
from gym_go.gogame import turn, random_weighted_action

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import matplotlib.pyplot as plt
import matplotlib
import numpy as np

# Constants

In [2]:
UCB_C = 30

# Board will be a BOARD_SIZE * BOARD_SIZE board (BOARD_SIZE**2)
BOARD_SIZE = 5

# 0 - Black pieces
# 1 - White pieces
# 2 - Turn (0 - black, 1 - white)
# 3 - Invalid moves (including ko-protection)
# 4 - Previous move was a pass
# 5 - Game over
BLACK, WHITE, INVALID = 0, 1, 3

# Monte Carlo Tree Search

1. Selection
    - Taverse the tree to find greatest UCB-score
2. Expansion
    - If the selected leaf node has been visited before expand by adding weighted game action
3. Rollout
    - Simulate the game until end-condition from the expanded leaf
4. Back-propagation
    - Updating the value of each ancestor node of the expanded leaf


In [3]:
# TODO: GymGo.gogame.move/validmove/elns for å nulle ut actionspace
def get_legal_move(env):
    return env.uniform_random_action()
    # board_shape = env.state().shape[1:]
    # pass_id = np.prod(board_shape)
    # action = env.action_space.sample() # pick random action
    # action2d = action // board_shape[0], action % board_shape[1], action
    # while action2d[2] != pass_id and env.state()[INVALID, action2d[0], action2d[1]] == 1:
    #     action = env.action_space.sample() # pick random action
    #     action2d = action // board_shape[0], action % board_shape[1], action
    # return action2d[2]

def equals(state_a, state_b):
    for i in range(0, 6):
        if not np.array_equal(state_a[i], state_b[i]):
            return False
    return True

In [4]:
class Node():
    def __init__(self, env, parent, action):
        self.env : gym.Env = env # This env will be altered by the other player
        self.value : int = 0 # Value estimate
        self.trials : int = 0 # Number of trials for this node
        self.parent : Node = parent # Parent node of this node
        self.children : list[Node] = [] # List of children of this node
        self.action : int = action # The step action made by this node
    
    # calculate a Upper Confidence Bound
    def ucb(self, total_trials):
        return self.value + ( UCB_C * sqrt(log(total_trials) / self.trials) )
    
    # Add a new node to a leaf node
    def expansion(self, move_weights):
        if self.env.done:
            return
        board_shape = self.env.state().shape[1:]
        pass_id = np.prod(board_shape)
        for action in range(0, pass_id + 1):
            action2d = action // board_shape[0], action % board_shape[1]
            if action == pass_id or self.env.state()[3, action2d[0], action2d[1]] == 0:
                child_env = copy.deepcopy(self.env)
                child_env.step(action)
                child = Node(child_env, self, action)
                child.value = move_weights[action]
                self.children.append(child)

    # Simulate game from current move until end-condition returning the score
    def rollout(self, move_selection_method):
        if self.env.done:
            return self.env.reward()
        
        rollout_env = copy.deepcopy(self.env)
        rollout_result = 0
        done = False
        while not done:
            random_action = move_selection_method(rollout_env)
            _, reward, done, _ = rollout_env.step(random_action)
            rollout_result += reward
        return rollout_result

In [5]:
class Monte_Carlo_Tree_Search():
    def __init__(self, size, ml_model):
        self.env : gym.Env = gym.make('gym_go:go-v0', size=size, reward_method='heuristic')
        self.env.reset()
        self.number_of_trials : int = 0
        self.root = Node(self.env, None, None)
        self.ml_model = ml_model
    
    def get_move_weights(self, state):
        # Do something to get this from ml_model
        move_weights = np.ones(BOARD_SIZE ** 2 + 1)
        
        board_shape = state.shape[1:]
        for i in range(len(move_weights) - 1):
            action2d = i // board_shape[0], i % board_shape[1], i
            if state[INVALID, action2d[0], action2d[1]] == 1:
                move_weights[i] = 0.0

        return move_weights

    def get_weighted_move(self, env : gym.Env):
        move_weights = self.get_move_weights(env.state())
        return random_weighted_action(move_weights)

    # Update scores of all parent nodes after rollout
    def back_propagation(self, rollout_node: Node, rollout_result):
        current_node = rollout_node
        while current_node != None:
            current_node.trials += 1
            if turn(self.env.state()) == BLACK:
                current_node.value -= rollout_result
            if turn(self.env.state()) == WHITE:                
                current_node.value += rollout_result
            current_node = current_node.parent
        self.number_of_trials += 1
    
    # find and return the leaf node with the highest UCB-score 
    def selection(self):
        selected_child = self.root
        current_node = self.root
        while len(current_node.children) > 0:
            selected_child = current_node.children[0]
            current_best_ucb = -inf

            for child in current_node.children:
                if child.trials == 0:
                    return child

                child_ucb = child.ucb(self.number_of_trials)

                if child_ucb > current_best_ucb:
                    selected_child = child
                    current_best_ucb = child_ucb

            current_node = selected_child

        return selected_child
        
    def run(self, iterations):
        selected_node = self.root
        selected_node.expansion(self.get_move_weights(selected_node.env.state()))
        selected_node = self.root.children[0]

        run = 0
        while run < iterations:
            selected_node = self.selection()
            # print("Run:", run, ": Selection", selected_node.action)

            if selected_node.env.done:
                self.back_propagation(selected_node, selected_node.env.reward())
                run += 1
                continue

            if selected_node.trials > 0:
                # print("Run:", run, ": Expansion:")
                selected_node.expansion(self.get_move_weights(selected_node.env.state()))
                selected_node = selected_node.children[0]

            rollout_result = selected_node.rollout(self.get_weighted_move)
            # print("Run:", run, ": Rollout", rollout_result)
            
            self.back_propagation(selected_node, rollout_result)
            run += 1
    
    def find_node_from_state(self, node: Node, state):
        if equals(node.env.state(), state):
            return node

        for child in node.children:
            if equals(child.env.state(), state):
                return child
            
            res = self.find_node_from_state(child, state)
            if res != None and equals(res.env.state(), state):
                return res

    def get_move_from_env(self, env):
        node = self.find_node_from_state(self.root, env.state())
        
        if node != None:
            print("Found node for state:")
            node.env.render()
            print("Finding best move from:", len(node.children), "available explored moves.")
            best_child = None
            current_best_value = -inf
            for child in node.children:
                if child.value > current_best_value:
                    best_child = child
                    current_best_value = child.value

            if best_child != None:
                return best_child
        
        return None

    def get_training_data_from_tree(self):
        training_data = []
        self.get_training_data_from_node(training_data, self.root)
        return training_data
        
    def get_training_data_from_node(self, training_data : list, current_node):
        node_tuple = [current_node.env.state(), [0] * (BOARD_SIZE ** 2 + 1)]
        for child in current_node.children:
            node_tuple[1][child.action] = child.value
            training_data.append(node_tuple)
            self.get_training_data_from_node(training_data, child)

    def get_tree_data(self):
        x, y = [], []
        self.__get_node_data(self.root, x, y)
        return torch.tensor(x), torch.tensor(y)

    def __get_node_data(self, node, x, y):
        x.append(node.env.state())
        y.append([0] * (BOARD_SIZE**2 + 1))
        for child in node.children:
            y[-1][child.action] = child.value
            self.__get_node_data(child, x, y)

In [6]:
model = Monte_Carlo_Tree_Search(5, None)
model.run(100)

c:\Users\erik\miniconda3\envs\ml-project\lib\site-packages\gym\utils\passive_env_checker.py:219: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(


In [7]:
# Sjekker hvor utforska treet er for å teste UCB_C konstanten

def print_tried_children(current_node : Node):
    for child in current_node.children:
        if child.trials == 0:
            continue
        print("Child", child.action, ":")
        print("Number of children:", len(child.children))
        print("Value:", child.value, "Number of trials:", child.trials, "UCB:", child.ucb(model.number_of_trials))

current_node = model.root
while len(current_node.children) > 0:
    most_tried = current_node.children[0]
    for child in current_node.children:
        if child.trials == 0:
            continue
        print("Child", child.action, ":")
        child.env.render()
        print("Number of children:", len(child.children))
        print("Value:", child.value, "Number of trials:", child.trials, "UCB:", child.ucb(model.number_of_trials))
        print_tried_children(child)
        if child.trials > most_tried.trials:
            most_tried = child
        current_node = most_tried

Child 0 :
	0 1 2 3 4 
0	○═╤═╤═╤═╗
1	╟─┼─┼─┼─╢
2	╟─┼─┼─┼─╢
3	╟─┼─┼─┼─╢
4	╚═╧═╧═╧═╝
	Turn: WHITE, Game State (ONGOING|PASSED|END): ONGOING
	Black Area: 25, White Area: 0

Number of children: 0
Value: 147.0 Number of trials: 1 UCB: 211.37898078868042
Child 1 :
	0 1 2 3 4 
0	╔═○═╤═╤═╗
1	╟─┼─┼─┼─╢
2	╟─┼─┼─┼─╢
3	╟─┼─┼─┼─╢
4	╚═╧═╧═╧═╝
	Turn: WHITE, Game State (ONGOING|PASSED|END): ONGOING
	Black Area: 25, White Area: 0

Number of children: 0
Value: 38.0 Number of trials: 1 UCB: 102.37898078868042
Child 2 :
	0 1 2 3 4 
0	╔═╤═○═╤═╗
1	╟─┼─┼─┼─╢
2	╟─┼─┼─┼─╢
3	╟─┼─┼─┼─╢
4	╚═╧═╧═╧═╝
	Turn: WHITE, Game State (ONGOING|PASSED|END): ONGOING
	Black Area: 25, White Area: 0

Number of children: 0
Value: 220.0 Number of trials: 1 UCB: 284.37898078868045
Child 3 :
	0 1 2 3 4 
0	╔═╤═╤═○═╗
1	╟─┼─┼─┼─╢
2	╟─┼─┼─┼─╢
3	╟─┼─┼─┼─╢
4	╚═╧═╧═╧═╝
	Turn: WHITE, Game State (ONGOING|PASSED|END): ONGOING
	Black Area: 25, White Area: 0

Number of children: 0
Value: -38.0 Number of trials: 1 UCB: 26.378980788680423
Child 4 :

In [8]:
def play_game(advesary_function, model : Monte_Carlo_Tree_Search, go_env: gym.Env):
    go_env.reset()
    done = go_env.done
    turn_nr = 0
    while not done:
        action = advesary_function(go_env)
        _, _, done, _ = go_env.step(action)
        go_env.render('terminal')

        if done:
            continue

        node = model.get_move_from_env(go_env)
        action = get_legal_move(go_env)
        if node != None:
            action = node.action  
        _, _, done, _ = go_env.step(action)
        go_env.render('terminal')
        turn_nr += 1
        if turn_nr > 300:
            break

    if node != None:
        model.back_propagation(node, go_env.reward())

def play_game_no_render(advesary_function, model : Monte_Carlo_Tree_Search, go_env: gym.Env):
    go_env.reset()
    done = go_env.done
    turn_nr = 0
    while not done:
        action = advesary_function(go_env)
        _, _, done, _ = go_env.step(action)

        if done:
            continue

        node = model.get_move_from_env(go_env)
        action = get_legal_move(go_env)
        if node != None:
            action = node.action  
        _, _, done, _ = go_env.step(action)
        turn_nr += 1
        if turn_nr > 300:
            break

    if node != None:
        model.back_propagation(node, go_env.reward())
    
    return go_env

def play_model_vs_model(model1 : Monte_Carlo_Tree_Search, model2 : Monte_Carlo_Tree_Search, go_env: gym.Env):
    go_env.reset()
    done = go_env.done
    turn_nr = 0
    while not done:
        node1 = model1.get_move_from_env(go_env)
        action = get_legal_move(go_env)
        if node1 != None:
            action = node1.action  
        _, _, done, _ = go_env.step(action)
        go_env.render('terminal')

        if done:
            continue

        node2 = model2.get_move_from_env(go_env)
        action = get_legal_move(go_env)
        if node2 != None:
            action = node2.action  
        _, _, done, _ = go_env.step(action)
        go_env.render('terminal')
        turn_nr += 1
        if turn_nr > 300:
            break

    if node1 != None:
        model1.back_propagation(node1, go_env.reward())
    if node2 != None:
        model2.back_propagation(node2, go_env.reward())

def play_model_vs_model_no_render(model1 : Monte_Carlo_Tree_Search, model2 : Monte_Carlo_Tree_Search, go_env: gym.Env):
    go_env.reset()
    done = go_env.done
    turn_nr = 0
    while not done:
        node1 = model1.get_move_from_env(go_env)
        action = get_legal_move(go_env)
        if node1 != None:
            action = node1.action  
        _, _, done, reward = go_env.step(action)

        if done:
            continue

        node2 = model2.get_move_from_env(go_env)
        action = get_legal_move(go_env)
        if node2 != None:
            action = node2.action  
        _, _, done, reward = go_env.step(action)
        turn_nr += 1
        if turn_nr > 300:
            break
    return go_env

In [9]:
for i in range(1,10):
    env = play_game_no_render(get_legal_move, model, copy.deepcopy(model.env))
    if env.done:
        print("Game finished within 300 turns:")
    else:
        print("Game stopped after 300 turns:")
        
    if env.reward() < 0:
        print("White won!")
    if env.reward() > 0:
        print("Black won!")
    if env.reward() == 0:
        print("It's a draw!")

Found node for state:
	0 1 2 3 4 
0	╔═╤═╤═╤═╗
1	╟─┼─┼─┼─╢
2	╟─┼─┼─○─╢
3	╟─┼─┼─┼─╢
4	╚═╧═╧═╧═╝
	Turn: WHITE, Game State (ONGOING|PASSED|END): ONGOING
	Black Area: 25, White Area: 0

Finding best move from: 0 available explored moves.
Game finished within 300 turns:
Black won!
Found node for state:
	0 1 2 3 4 
0	╔═╤═╤═○═╗
1	╟─┼─┼─┼─╢
2	╟─┼─┼─┼─╢
3	╟─┼─┼─┼─╢
4	╚═╧═╧═╧═╝
	Turn: WHITE, Game State (ONGOING|PASSED|END): ONGOING
	Black Area: 25, White Area: 0

Finding best move from: 0 available explored moves.


c:\Users\erik\miniconda3\envs\ml-project\lib\site-packages\gym\utils\passive_env_checker.py:219: DeprecationWarning: WARN: Core environment is written in old step API which returns one bool instead of two. It is recommended to rewrite the environment with new step API. 
  logger.deprecation(


Game finished within 300 turns:
Black won!
Found node for state:
	0 1 2 3 4 
0	╔═╤═╤═╤═○
1	╟─┼─┼─┼─╢
2	╟─┼─┼─┼─╢
3	╟─┼─┼─┼─╢
4	╚═╧═╧═╧═╝
	Turn: WHITE, Game State (ONGOING|PASSED|END): ONGOING
	Black Area: 25, White Area: 0

Finding best move from: 0 available explored moves.
Game finished within 300 turns:
White won!
Found node for state:
	0 1 2 3 4 
0	╔═╤═╤═╤═╗
1	╟─┼─┼─┼─╢
2	╟─┼─┼─┼─○
3	╟─┼─┼─┼─╢
4	╚═╧═╧═╧═╝
	Turn: WHITE, Game State (ONGOING|PASSED|END): ONGOING
	Black Area: 25, White Area: 0

Finding best move from: 0 available explored moves.
Game finished within 300 turns:
White won!
Found node for state:
	0 1 2 3 4 
0	╔═╤═╤═╤═╗
1	╟─┼─┼─┼─╢
2	╟─┼─┼─┼─╢
3	○─┼─┼─┼─╢
4	╚═╧═╧═╧═╝
	Turn: WHITE, Game State (ONGOING|PASSED|END): ONGOING
	Black Area: 25, White Area: 0

Finding best move from: 0 available explored moves.
Game finished within 300 turns:
Black won!
Found node for state:
	0 1 2 3 4 
0	╔═╤═╤═╤═╗
1	╟─┼─┼─┼─╢
2	○─┼─┼─┼─╢
3	╟─┼─┼─┼─╢
4	╚═╧═╧═╧═╝
	Turn: WHITE, Game State (ONGOING|P

# Convolutional Neural Network

In [10]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using {device}")

Using cuda:0


In [11]:
class CNN(nn.Module):

    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(6, 32, 5, padding=2)
        self.pool = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(32, 64, 5, padding=2)
        self.fc1 = nn.Linear(64, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 26)


    def forward(self, x):
        x = x.float()
        x = x.to(device)
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x,0, -1)
        x = x.t()
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def accuracy(self, x, y):
        return torch.mean(torch.eq(self.forward(x).argmax(1), y.argmax(1)).float())

In [12]:
# utility methods for importing and exporting models, defined outside class
def export_model(cnn, name="cnn"):
    torch.save(cnn.state_dict(), "cnns/" + name + ".pth")

def import_model(cnn, name="cnn"):
    cnn.load_state_dict(torch.load("cnns/" + name + ".pth"))

In [13]:
def train_model(model, x, y, lr, momentum):
    print(f"Training on {len(x)} datapoints")

    model.to(device)

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

    running_loss = .0

    for i in range(len(x)):
        inputs, labels = torch.tensor(x[i], device=device), torch.tensor(y[i], device=device)
        # inputs = torch.permute(inputs, (2, 1, 0))
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

        if i % 200 == 199:
            print(f'[{i + 1:8d}] loss: {running_loss / 200:.3f}')
            running_loss = 0.0

# MCTS and CNN combined

1. Create an MCTS
2. Generate a lot of data from the MCTS
3. Use the data to train a CNN
4. Use the CNN in a new MCTS to hopefully make the MCTS make better choices
5. Generate new data with the new MCTS
6. Train a new CNN... repeat ad infinitum

In [14]:
cnn = CNN()
mcts = Monte_Carlo_Tree_Search(BOARD_SIZE, cnn)
mcts.run(10_000)
x, y = mcts.get_tree_data()
print(f"Model structure: {cnn}\n\n")

# for name, param in cnn.named_parameters():
#     print(f"Layer: {name} | Size: {param.size()} \n")
#     # print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: CNN(
  (conv1): Conv2d(6, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (fc1): Linear(in_features=64, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=26, bias=True)
)




In [15]:
cnn = CNN()
train_model(cnn, x, y, .001, .9)
print(cnn.accuracy(x, y))
# go_env = gym.make()

Training on 1174 datapoints
[     200] loss: nan
[     400] loss: nan
[     600] loss: nan
[     800] loss: nan
[    1000] loss: nan


RuntimeError: mat1 and mat2 shapes cannot be multiplied (1x75136 and 64x120)